In [ ]:
import pandas as pd
import scipy

import netpixi
import graph_tool_extras as gte

import distribution as dst
from graph_tool import centrality

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
g = gte.load('./rede.net.gz')

# Descrição da Rede
Obtendo o tamanho da rede (número de vértices e arestas).

In [ ]:
print(f'Número de Verticies: {g.num_vertices()}')
print(f'Número de Arestas: {g.num_edges()}')

Obtendo a densidade da rede por meio da função `density`.

In [ ]:
print(f'Densidade: {g.density()}%')

Obtendo a transitividade da rede por meio da função `transitivity`.

In [ ]:
print(f'Transitividade: {g.transitivity()}%')

Obtendo o número total de vizinhos de cada um dos vértic por meio da função `get_total_degrees`.  

In [ ]:
degrees = g.get_total_degrees()

In [ ]:
degrees.describe()

In [ ]:
degrees.hist(bins=50, edgecolor='white', linewidth=0.5);

Analizando a **normalidade** da distribuição dos `degree` através da função `not_normal`.

In [ ]:
p_value = dst.not_normal(degrees)
print(f'Not Normal: {'<0.001' if p_value < 0.001 else float(p_value)}')

Analizando se a distribuição dos `degree` é mais **power law** do que **lognormal** através da função `more_powerlaw_than_lognormal`.

In [ ]:
powerlaw, p_value = dst.more_powerlaw_than_lognormal(degrees)
print(f'More Powerlaw than Lognormal: {'{float(p_value)} (Inconclusivo)' if 0.05 < p_value < 0.1 else ('<0.001' if  p_value < 0.001 else float(p_value))} {'[powerlaw]' if powerlaw else ''}')

Analizando se a destribuição dos `degree` é mais **power law** do que **exponential** através da função `more_powerlaw_than_exponential`.

In [ ]:
powerlaw, p_value = dst.more_powerlaw_than_exponential(degrees)
print(f'More Powerlaw than Exponential: {'{float(p_value)} (Inconclusivo)' if 0.05 < p_value < 0.1 else ('<0.001' if  p_value < 0.001 else float(p_value))} {'[powerlaw]' if powerlaw else ''}')

## Distâncias

Obtendo as distâncias através do método `describe_distances` e plotando o histograma com `hist_distances`.

**OBS.:** As funções `get_distances`, `hist` e `describe` não foram utilizadas pois o _kernel_ tende a crashar antes que elas pudessem finalizar a execução.

In [ ]:
g.describe_distances()

In [ ]:
g.hist_distances()

# Métricas da Rede

## Closeness
Primeiro, calculamos a centralidade _closeness_ com `closeness`.

In [ ]:
c = centrality.closeness(g, harmonic=True)

Depois, adicionamos a propriedade de vértice ao grafo com `add_vp`.

In [ ]:
g.add_vp('closeness', c)

## Betweeness
Agora, para calcularmos o _betweeness_, utilizamos a função `betweenness`.

In [ ]:
b, _ = centrality.betweenness(g)

Novamente, precisamos adicionar a propriedade de vértice ao grafo com `add_vp`.

In [ ]:
g.add_vp('betweenness', b)

## Degrees
Para calcular a centralidade _degree_, utilizamos a função `total_degree`.

In [ ]:
d = gte.total_degree(g)

Adicionamos a propriedade ao grafo.

In [ ]:
g.add_vp('degree', d)

## Visualização da Rede
Agora com todas as 3 métricas calculadas podemos salvar e visualiza-las na rede.

In [ ]:
gte.save(g, 'rede_metricas.net.gz')

In [ ]:
r = netpixi.render('rede_metricas.net.gz', infinite=True)

Primeiro, mas não menos importante, a visualização padrão da rede, sem a aplicação de nenhuma métrica

In [ ]:
r.vertex_default(size=4, bwidth=1)
r.edge_default(width=1)

**OBS.:** Descomente as mudanças de escala para ver o resultado de cada métrica.

In [ ]:
# r.vertex_scale('closeness', 10, 30)
# r.vertex_scale('betweenness', 10, 30)
# r.vertex_scale('degree', 10, 30)

## Gráficos

Nessa seção, faremos o _plot_ dos gráficos relacionando _closeness_, _betweenness_ e _degree_, seguindo a ordem apresentada na planilha de entrega da atividade.

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(16, 10))

fig.suptitle("Relação entre as propriedades da rede")

sns.histplot(c, ax=axes[0, 0]).set_title("Closeness")
sns.scatterplot(x=b, y=c, ax=axes[0, 1]).set_title("Betweenness X Closeness")
sns.scatterplot(x=d, y=c, ax=axes[0, 2]).set_title("Degree X Closeness")

sns.scatterplot(x=c, y=b, ax=axes[1, 0]).set_title("Closeness X Betweeness")
hist_betweenness = sns.histplot(b, ax=axes[1, 1])
hist_betweenness.set_title("Betweenness")
hist_betweenness.set_ylim(0, 1000)
sns.scatterplot(x=d, y=b, ax=axes[1, 2]).set_title("Degree X Betweeness")

sns.scatterplot(x=c, y=d, ax=axes[2, 0]).set_title("Closeness X Degree")
sns.scatterplot(x=b, y=d, ax=axes[2, 1]).set_title("Betweeness X Degree")
sns.histplot(d, ax=axes[2, 2]).set_title("Degree")

plt.show()

A célula abaixo possui código para baixar imagens dos gráficos acima, separadamente. Descomentar caso necessário.

In [ ]:
# import os
# if not os.path.isdir('img'):
#     os.mkdir('img')

# sns.histplot(c).get_figure().savefig('./img/hist_closeness.png')
# plt.close()
# sns.scatterplot(x=b, y=c).get_figure().savefig('./img/betweenness_closeness.png')
# plt.close()
# sns.scatterplot(x=d, y=c).get_figure().savefig('./img/degree_closeness.png')
# plt.close()

# sns.scatterplot(x=c, y=b).get_figure().savefig('./img/closeness_betweenness.png')
# plt.close()
# hist_betweenness = sns.histplot(b)
# hist_betweenness.set_ylim(0, 1000)
# hist_betweenness.get_figure().savefig('./img/hist_betweenness.png')
# plt.close()
# sns.scatterplot(x=d, y=b).get_figure().savefig('./img/degree_betweenness.png')
# plt.close()

# sns.scatterplot(x=c, y=d).get_figure().savefig('./img/closeness_degree.png')
# plt.close()
# sns.scatterplot(x=b, y=d).get_figure().savefig('./img/betweenness_degree.png')
# plt.close()
# sns.histplot(d).get_figure().savefig('./img/hist_degree.png')
# plt.close()

## Correlações
Para calcular as correlações utilizamos a função `pearsonr` com combinaçãos dos valores de `closeness`, `betweeness` e `degree`, obtendo um coeficiente e um p-valor, este que será convertido em uma escala de asteriscos.

In [ ]:
def pearsonr(x, y, ndigits=10):
    statistic, p_value = scipy.stats.pearsonr(x, y)
    
    asterisk = ''
    statistic = round(statistic, ndigits)
    
    if statistic == 1: 
        asterisk = ' (óbvio)'
    elif p_value < 0.01: 
        asterisk = ' ***'
    elif p_value < 0.05: 
        asterisk = ' **'
    elif p_value < 0.1: 
        asterisk = ' *'
        
    return f'{statistic} {asterisk}'

In [ ]:
data = [c, b, d]
result = pd.DataFrame([ [ pearsonr(j, i) for j in data ] for i in data ])
result